# Impepdom Test Set Performance Analyses


In [1]:
import os
import sys
sys.path.append("..")  # add top folder to path
from collections import Counter

import numpy as np
import pandas as pd
import scipy.stats as sts
import matplotlib.pyplot as plt
import torch

import impepdom

## Model Training

In [57]:
'''
Should make this whole block as functions inside Impepdom package later
'''
def weighted_harmonic_mean(var_1, var_2, beta=1):  # should make it generalizable to many variables
    '''
    Harmonic mean for two parameters with weighting.
    
    Parameters
    ----------
    var_1, var_2: int or ndarray
        Variables to consider
        
    beta: float, optional
        Importance of `var_2` relative to `var_1`.
        If beta == 1, this function is equivalent to `scipy.stats.hmean()`
    '''
    
    return (1 + beta**2) * np.multiply(var_1, var_2) / (beta**2 * var_1 + var_2)

def get_best_hyperparams(file, padding='end'):
    '''
    Extract the best hyperparameters for a model with harmonic weighting

    Parameters
    ----------
    file: string
        Name of the CSV file contraining hyperparam results
    '''

    path = '../store/hyperparams'

    file_end = file.find('.csv')
    file_begin = max(file.find('mlp'), file.find('cnn'))
    all_name = file[file_end-6:file_end-2] + (':' if file.find(':') == -1 else '') + file[file_end-2:file_end]
    allele = 'HLA-' + all_name.upper() # change to appropriate name

    df = pd.read_csv(path + '/' + file)
    correct_padding = (df['padding'] == padding)
    df = df[correct_padding].reset_index()
    idx = (df['min_auc'].notna() & df['mean_ppv'].notna())
    
    metric_1, metric_2 = np.array(df['min_auc'][idx]), np.array(df['mean_ppv'][idx])
    beta = 1  # how much the second metric should be weighted compared to the first
    w_hmean = weighted_harmonic_mean(metric_1, metric_2, beta=0.6)
    
    best_3_rows = (-w_hmean).argsort()[:3] # for top 3 rows with best harmonic mean value
    # best_3_rows = (-sts.hmean([df['min_auc'][idx], df['mean_ppv'][idx]])).argsort()[:3]
    
    batch_sizes = list(df['batch_size'][best_3_rows].astype('int'))
    batch_counter = Counter(batch_sizes)
    batch_sz = batch_counter.most_common(1)[0][0]
    
    hyperparams = {
        'hla_allele': allele, 
        'padding': padding,
        'batch_size': batch_sz, 
        'num_epochs': int(np.mean(df['num_epochs'][best_3_rows])),
        'learning_rate': float(np.mean(df['learning_rate'][best_3_rows])),

        'min_auc': list(metric_1[best_3_rows]),
        'mean_ppv': list(metric_2[best_3_rows]),
    }

    if 'mean_pcc' in df.columns:
        hyperparams['mean_pcc'] = list(df['mean_pcc'][best_3_rows])
    if 'dropout_input' in df.columns and 'dropout_hidden' in df.columns:
        hyperparams['dropout_input'] = float(np.mean(df['dropout_input'][best_3_rows]))
        hyperparams['dropout_hidden'] = float(np.mean(df['dropout_hidden'][best_3_rows]))
    else:
        hyperparams['dropout_input'] = 0.65  # stolen from MLP end
        hyperparams['dropout_hidden'] = 0.46
        
    hyperparams['conv'] = False
    if 'conv' in df.columns:
        if df['conv'][0] == 'True':
            hyperparams['conv'] = True
            
    if 'num_conv_layers' in df.columns and 'conv_filt_sz' in df.columns	and 'conv_stride' in df.columns:
        hyperparams['num_conv_layers'] = int(np.max(df['num_conv_layers'][best_3_rows]))
        hyperparams['conv_filt_sz'] = int(np.max(df['conv_filt_sz'][best_3_rows]))
        hyperparams['conv_stride'] = int(np.max(df['conv_stride'][best_3_rows]))
    else:
        hyperparams['num_conv_layers'] = 1  # default params for conv
        hyperparams['conv_filt_sz'] = 5
        hyperparams['conv_stride'] = 2

    return hyperparams

def make_trained_model(hyperparams):
    results = []

    impepdom.time_tracker.reset_timer() 

    print('working with allele', hyperparams['hla_allele'])
    dataset = impepdom.PeptideDataset(
        hla_allele=hyperparams['hla_allele'],
        padding=hyperparams['padding'],
        toy=False)

    save_folder, baseline_metrics, _ = impepdom.run_experiment(
        model_type='MultilayerPerceptron',  # passing model type here
        dataset=dataset,
        train_fold_idx=[0, 1, 2, 3, 4],
        learning_rate=hyperparams['learning_rate'],
        num_epochs=hyperparams['num_epochs'],
        batch_size=hyperparams['batch_size'],

        show_output=True,
        dropout_input=hyperparams['dropout_input'],
        dropout_hidden=hyperparams['dropout_hidden'],

        conv=hyperparams['conv'],
        num_conv_layers=hyperparams['num_conv_layers'],
        conv_filt_sz=hyperparams['conv_filt_sz'],
        conv_stride=hyperparams['conv_stride']
    )

    model = impepdom.models['MultilayerPerceptron'](
        dropout_input=hyperparams['dropout_input'],
        dropout_hidden=hyperparams['dropout_hidden'],

        conv=hyperparams['conv'],
        num_conv_layers=hyperparams['num_conv_layers'],
        conv_filt_sz=hyperparams['conv_filt_sz'],
        conv_stride=hyperparams['conv_stride']
    )
    
    # to load a trained model, you would need to initialize the model outside
    # smth like (untrained) `model = impepdom.models.MultilayerPerceptron(num_hidden_layers=2, hidden_layer_size=100)`
    # but specify params exactly the same as in hyperparam_search
                                                                    # get from csv file (column "model")
    trained_model, _ = impepdom.load_trained_model(model, save_folder)

    return trained_model, save_folder

## Prediction on Test Set

In [ ]:
def 

## I. Impepdom MLP, Padding End (A01:01)

In [39]:
hyp_mlp_end = get_best_hyperparams('mlp_2x100_a01:01.csv', padding='end')
hyp_mlp_end

{'hla_allele': 'HLA-A01:01',
 'padding': 'end',
 'batch_size': 32,
 'num_epochs': 10,
 'learning_rate': 0.001,
 'min_auc': [0.8977589504924592, 0.8961656106026613, 0.8990260131998715],
 'mean_ppv': [0.5537177398133175, 0.5552848410310801, 0.5488537884133244],
 'mean_pcc': [0.6042904818034925, 0.6009710200191343, 0.6021092651503497],
 'dropout_input': 0.65,
 'dropout_hidden': 0.4666666666666666,
 'conv': False,
 'num_conv_layers': 2,
 'conv_filt_sz': 5,
 'conv_stride': 1}

In [40]:
trained_mlp_end, save_folder_mlp_end = make_trained_model(hyp_mlp_end)

working with allele HLA-A01:01
[1 m 12 s] peptide dataset initialized
epoch 1/10 started at 0.0005 s
train loss: 0.0366 acc: 0.9917 auc: 0.8827

epoch 2/10 started at 76.0032 s
train loss: 0.0339 acc: 0.9926 auc: 0.8911

epoch 3/10 started at 149.1546 s
train loss: 0.0329 acc: 0.9928 auc: 0.8922

epoch 4/10 started at 221.5307 s
train loss: 0.0313 acc: 0.9932 auc: 0.8967

epoch 5/10 started at 294.6287 s
train loss: 0.0321 acc: 0.9932 auc: 0.8904

epoch 6/10 started at 368.3679 s
train loss: 0.0313 acc: 0.9933 auc: 0.8961

epoch 7/10 started at 443.1616 s
train loss: 0.0325 acc: 0.9930 auc: 0.8905

epoch 8/10 started at 518.5411 s
train loss: 0.0312 acc: 0.9932 auc: 0.8932

epoch 9/10 started at 594.3561 s
train loss: 0.0314 acc: 0.9932 auc: 0.8917

epoch 10/10 started at 670.5593 s
train loss: 0.0317 acc: 0.9932 auc: 0.8860

training completed in 12 m 35.4826 s
best training auc: 0.8967


## II. Impepdom MLP, Padding Flurry (A01:01)

In [58]:
hyp_mlp_flurry = get_best_hyperparams('flurry_mlp_2x100_a01:01.csv', padding='flurry')
hyp_mlp_flurry

{'hla_allele': 'HLA-A01:01',
 'padding': 'flurry',
 'batch_size': 32,
 'num_epochs': 2,
 'learning_rate': 0.005,
 'min_auc': [0.872914348179539, 0.8887562782268924, 0.8829656080326539],
 'mean_ppv': [0.702036238962418, 0.6725481692381067, 0.6724674239309099],
 'dropout_input': 0.65,
 'dropout_hidden': 0.46,
 'conv': False,
 'num_conv_layers': 1,
 'conv_filt_sz': 5,
 'conv_stride': 2}

In [59]:
trained_mlp_flurry, save_folder_mlp_flurry = make_trained_model(hyp_mlp_flurry)

working with allele HLA-A01:01
[1 m 18 s] peptide dataset initialized
epoch 1/2 started at 0.0006 s
train loss: 0.0367 acc: 0.9921 auc: 0.8990

epoch 2/2 started at 80.6436 s
train loss: 0.0360 acc: 0.9923 auc: 0.9101

training completed in 2 m 38.5692 s
best training auc: 0.9101


## III. Impepdom CNN, Padding Flurry (A01:01)

In [30]:
hyp_cnn_flurry = get_best_hyperparams('mlp_2x100_cnn_a01:01.csv', padding='flurry')
hyp_cnn_flurry

{'hla_allele': 'HLA-A01:01',
 'padding': 'flurry',
 'batch_size': 32,
 'num_epochs': 6,
 'learning_rate': 0.0007,
 'min_auc': [0.9343186974226512, 0.930497976816677, 0.935205387103132],
 'mean_ppv': [0.5999044863447609, 0.6009233348300029, 0.5940511290114937],
 'mean_pcc': [0.6569269182965304, 0.6589301105293429, 0.6550037584822265],
 'dropout_input': 0.16666666666666666,
 'dropout_hidden': 0.4166666666666667,
 'conv': True,
 'num_conv_layers': 2,
 'conv_filt_sz': 5,
 'conv_stride': 1}

In [31]:
trained_cnn_flurry, save_folder_cnn_flurry = make_trained_model(hyp_cnn_flurry)

working with allele HLA-A01:01
[1 m 16 s] peptide dataset initialized
epoch 1/6 started at 0.0008 s
train loss: 0.0303 acc: 0.9930 auc: 0.9142

epoch 2/6 started at 295.2853 s
train loss: 0.0265 acc: 0.9937 auc: 0.9341

epoch 3/6 started at 600.7621 s
train loss: 0.0257 acc: 0.9938 auc: 0.9426

epoch 4/6 started at 995.9847 s
train loss: 0.0248 acc: 0.9941 auc: 0.9441

epoch 5/6 started at 1340.9808 s
train loss: 0.0250 acc: 0.9939 auc: 0.9463

epoch 6/6 started at 1683.5154 s
train loss: 0.0241 acc: 0.9942 auc: 0.9516

training completed in 33 m 45.9678 s
best training auc: 0.9516


## IV. Impepdom CNN, Padding End (A01:01)

In [28]:
hyp_cnn_end = get_best_hyperparams('mlp_2x100_cnn_a01:01.csv', padding='end')
hyp_cnn_end

{'hla_allele': 'HLA-A01:01',
 'padding': 'end',
 'batch_size': 32,
 'num_epochs': 4,
 'learning_rate': 0.0007,
 'min_auc': [0.9368633953874179, 0.9313492851709656, 0.9336545279088934],
 'mean_ppv': [0.5958793626912142, 0.597742716862056, 0.5931293774163101],
 'mean_pcc': [0.6562752247286305, 0.6550852579461708, 0.6530620180032013],
 'dropout_input': 0.18333333333333335,
 'dropout_hidden': 0.45,
 'conv': True,
 'num_conv_layers': 2,
 'conv_filt_sz': 5,
 'conv_stride': 1}

In [29]:
trained_cnn_end, save_folder_cnn_end = make_trained_model(hyp_cnn_end)

working with allele HLA-A01:01
[1 m 15 s] peptide dataset initialized
epoch 1/4 started at 0.0006 s
train loss: 0.0317 acc: 0.9929 auc: 0.9056

epoch 2/4 started at 308.7786 s
train loss: 0.0260 acc: 0.9939 auc: 0.9386

epoch 3/4 started at 627.1995 s
train loss: 0.0254 acc: 0.9938 auc: 0.9438

epoch 4/4 started at 950.0328 s
train loss: 0.0245 acc: 0.9941 auc: 0.9478

training completed in 21 m 36.2425 s
best training auc: 0.9478


In [61]:
dataset = impepdom.PeptideDataset(
    hla_allele='HLA-A01:01',
    padding='end',
    toy=False)

[8 m 14 s] peptide dataset initialized


In [62]:
dataset.get_fold()

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0., 0., 0., ..., 0., 0., 0.]))

## V. Impepdom CNN, Padding End (B08:01)

In [60]:
hyp_cnn_end_b0801 = get_best_hyperparams('mlp_2x100_cnn_b08:01.csv', padding='end')
hyp_cnn_end_b0801

{'hla_allele': 'HLA-B08:01',
 'padding': 'end',
 'batch_size': 32,
 'num_epochs': 9,
 'learning_rate': 0.001,
 'min_auc': [0.9529939536379968, 0.961533306486236, 0.9580878946911502],
 'mean_ppv': [0.7607938897458618, 0.7426148757544642, 0.7399735726818328],
 'mean_pcc': [0.7852789903522068, 0.7810577608677632, 0.773902509671378],
 'dropout_input': 0.31666666666666665,
 'dropout_hidden': 0.6166666666666667,
 'conv': False,
 'num_conv_layers': 2,
 'conv_filt_sz': 5,
 'conv_stride': 1}

In [ ]:
trained_cnn_end_b0801, save_folder_cnn_end_b0801 = make_trained_model(hyp_cnn_end_b0801)